In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#path to the 'DataGenerator.py' and the 'DenseNet3D.py' files
%cd /content/drive/My Drive/gitRendu

#PATH variables (don't forget the last '/')
framesPATH = '/content/drive/My Drive/gitRendu/data/frames/' #where are stored the Frames
masksPATH = '/content/drive/My Drive/gitRendu/data/masks/' #where are stored the masks
weightsPATH = '/content/drive/My Drive/gitRendu/data/weights/' #where will be saved the model

#training variables
nbrClasses = 4 #total number of classes in the masks (counting '0' as a class)
initialImageShape = (352, 512, 35) #what was the original image shape
modelName = 'exampleModel' #the name of the model that will be loaded

/content/drive/My Drive/gitRendu


In [3]:
!pip install h5py
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-z68uqjjd
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-z68uqjjd
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=54b87913b9b37c7204fabb451859dc2f4aa1ac330e747999672a8c2e82b7e93b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3waxs1on/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
import numpy as np
import time
import glob, os
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot as plt
import keras
import tensorflow
import skimage
import sys
from skimage.transform import resize
from scipy.ndimage import zoom
import cv2
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from DataGenerator import DataGenerator
import DenseNet3D

Using TensorFlow backend.


In [5]:
model = DenseNet3D.DenseNet3D_FCN(input_shape=(80, 128, 32, 1),nb_dense_block=4, growth_rate=16,\
                       nb_layers_per_block=3, upsampling_type='upsampling', classes=nbrClasses, activation='softmax')

In [0]:
N = len(os.listdir(framesPATH))
#we compute the score for 'AIA terminal nodule', 'AIA central nodule' and 'other'(i.e. background, noise and other cells) => we have 3 classes
n_class = 3

AllID = np.arange(N)
np.random.shuffle(AllID)

F1_class0 = []
F1_class1 = []
F1_class2 = []

acc = []
acc_class0 = []
acc_class1 = []
acc_class2 = [] 

val_size = 20

In [7]:
conf_matrix = np.zeros((n_class,n_class))
model.load_weights(weightsPATH + modelName)

In [8]:
for ID in AllID[:val_size]:
  print(ID)
  x = np.load(framesPATH + 'frame_'+str(ID)+'.npy')
  x_pred = x.reshape((1,80, 128, 32,1))
  y_pred = model.predict(x_pred)
  y = np.load(masksPATH + 'mask_'+str(ID)+'.npy')
  y_pred = np.around(y_pred).reshape((80, 128, 32,4))
  y_pred_to_plot = np.zeros(np.shape(y_pred)[:3]) 
  for i in range(len(y_pred[:])):
    for j in range(len(y_pred[i])):
      for k in range(len(y_pred[i,j,:,:])):
        for c in range(len(y_pred[i,j,k,:])):
          if y_pred[i, j, k, c] > 0:
           y_pred_to_plot[i, j, k] = c
  #put 'noise' and cells that are not 'AIA terminal nodule' and 'AIA central nodule' as background
  y_pred_to_plot[np.where(y_pred_to_plot==3)] = 0
  y[np.where(y==3)] = 0
  y_pred_comp = resize(y_pred_to_plot, initialImageShape, mode='edge',anti_aliasing=False,anti_aliasing_sigma=None, order=0)
  y = resize(y, initialImageShape, mode='edge',anti_aliasing=False,anti_aliasing_sigma=None, order=0)
  conf_matrix = conf_matrix + confusion_matrix(np.reshape(y,(np.prod(initialImageShape),1)), np.reshape(y_pred_comp,(np.prod(initialImageShape),1)), labels = range(n_class) )
print(conf_matrix)
for i,line in enumerate(conf_matrix):
  acc.append(line[i]/sum(line))
acc_class0.append(acc[0])
acc_class1.append(acc[1])
acc_class2.append(acc[2])
TP = conf_matrix[0,0]
FP = sum(conf_matrix[1:,0])
FN = sum(conf_matrix[0,1:])
F1_class0.append(2*TP/(2*TP+FP+FN))
TP = conf_matrix[1,1]
FP = conf_matrix[1,0] + conf_matrix[1,2]
FN = conf_matrix[0,1] + conf_matrix[2,1]
F1_class1.append(2*TP/(2*TP+FP+FN))
TP = conf_matrix[2,2]
FP = conf_matrix[2,0] + conf_matrix[2,1]
FN = conf_matrix[0,2] + conf_matrix[1,2]
F1_class2.append(2*TP/(2*TP+FP+FN))

31
231
197
72
206
238
4
182
212
117
50
80
71
52
125
196
156
102
119
221
[[1.218333e+08 2.675120e+05 3.731448e+06]
 [6.012000e+03 1.912640e+05 6.920000e+02]
 [2.392000e+03 1.600000e+01 1.241640e+05]]


In [9]:
print(acc_class0)
print(acc_class1)
print(acc_class2)
print(F1_class0)
print(F1_class1)
print(F1_class2)

[0.968219914352647]
[0.966135941162208]
[0.9809752551907215]
[0.9838200029777857]
[0.5824471648699677]
[0.062348915708146574]


In [0]:
F1_class0 = []
F1_class1 = []
F1_class2 = []

acc = []
acc_class0 = []
acc_class1 = []
acc_class2 = [] 

for i,line in enumerate(conf_matrix):
  acc.append(line[i]/sum(line))
acc_class0.append(acc[0])
acc_class1.append(acc[1])
acc_class2.append(acc[2])
TP = conf_matrix[0,0]
FP = sum(conf_matrix[1:,0])
FN = sum(conf_matrix[0,1:])
F1_class0.append(2*TP/(2*TP+FP+FN))
TP = conf_matrix[1,1]
FP = conf_matrix[1,0] + conf_matrix[1,2]
FN = conf_matrix[0,1] + conf_matrix[2,1]
F1_class1.append(2*TP/(2*TP+FP+FN))
TP = conf_matrix[2,2]
FP = conf_matrix[2,0] + conf_matrix[2,1]
FN = conf_matrix[0,2] + conf_matrix[1,2]
F1_class2.append(2*TP/(2*TP+FP+FN))

In [11]:
print(acc_class0)
print(acc_class1)
print(acc_class2)
print(F1_class0)
print(F1_class1)
print(F1_class2)

[0.968219914352647]
[0.966135941162208]
[0.9809752551907215]
[0.9838200029777857]
[0.5824471648699677]
[0.062348915708146574]
